In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg
!pip3 install spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 782.7 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.1-py3-none-any.whl size=782936123 sha256=7974d1ba3745c234d1e6e9686781348a63d74870882269840f99cb04443d59f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-n69cg8h7/wheels/41/75/77/c4a98e18b2c317a2a13931cbbea7e3ca7f3a21efc36adc1d71
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Restart runtime after running above

In [ ]:
import spacy
import random

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from spacy.util import minibatch, compounding

## Dummy training data

In [ ]:
train_data = [
              ('WBC is a good bank', {'entities': [(0, 3, 'bank')]}),
              ('ABC is a good bank', {'entities': [(0, 3, 'bank')]})
]

## Configure pipeline when training from scratch

In [ ]:
# Create blank model
nlp = spacy.blank('en')

# Add NER pipeline
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

# Add labels to NER pipeline
ner = nlp.get_pipe('ner')
ner.add_label('bank')

## Create list of pipes to skip from training

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

## Train model

In [ ]:
with nlp.disable_pipes(*other_pipes):
    # Prepare optimizer
    optimizer = nlp.begin_training()

    epochs = 10
    for i in range(epochs):
        random.shuffle(train_data)
        losses = {}
        # Batch size start from 4 and keep increasing till 32
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            text, annotations = zip(*batch)
            nlp.update(text, 
                    annotations, 
                    sgd=optimizer, 
                    losses=losses,
                    drop=0.2)
        print(losses)

## Test model

In [ ]:
nlp('ABC is a nice bank compared to BOI').ents